In [1]:
import pandas as pd
import numpy as np

import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

import geopandas as gpd


In [2]:
maps = gpd.read_file('../data/sp_municipios.json')

mask = maps['nome_mesorregiao']=='Vale do Paraíba Paulista'

vale = maps[mask]

sp = maps[np.logical_not(mask)].dissolve(by='uf', as_index=False)

sp_vale = pd.concat([vale,sp],axis=0)
sp_vale = gpd.GeoDataFrame(sp_vale)
sp_vale.crs = {'init' :'epsg:4326'}

/home/m/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [3]:
df = pd.read_excel('../data/emendas_impositivas.xlsx')
mask = df['LONGITUDE LATITUDE']!="ESCOLAS A, DEFINIR "
df = df[mask]

df['value'] = (df['VALOR']/1000).astype(int).astype(str) + " mil"

df['LOCAL'] = df['LOCAL'].str.upper()
df['BENEFICIÁRIO'] = df['BENEFICIÁRIO'].str.upper()





# df['FINALIDADE'] = df['FINALIDADE'].str.replace('•',"")
# df['FINALIDADE'] = df['FINALIDADE'].str.replace('•',"<br> <br> •") 

df['lat'] = df['LONGITUDE LATITUDE'].str.replace("@",'').apply(lambda x: x.split(",")[0])
df['lng'] = df['LONGITUDE LATITUDE'].str.replace("@",'').apply(lambda x: x.split(",")[1])

locations = df[['lat', 'lng']]
locationlist = locations.values.tolist()

In [4]:
# df.to_excel('emendas.xlsx', index=False, encoding='utf-8')

In [6]:

map2 = folium.Map(location=[-23.061443,-45.8838777], zoom_start=8, tiles = 'cartodbpositron', min_zoom=7 ,max_zoom=18)

# marker_cluster = MarkerCluster(overlay=False,control=False, show=False).add_to(map2)

municipio = df['LOCAL'].tolist()
valor     = df['value'].tolist()
beneficiario  = df['BENEFICIÁRIO'].tolist()
finalidade  = df['FINALIDADE'].tolist()



for i in range(0, len(locationlist)):
    

    text = "<b>Municipio:</b> " + str(municipio[i]) \
    + '<br>''<br>'+ "<b>Entidade:</b> "+ str(beneficiario[i]) \
    + '<br>''<br>' "<b>Valor:</b> "+ str(valor[i]) \
    + '<br>''<br>' "<b>Finalidade:</b> "+ str(finalidade[i]) 
    
    size=1.2
    popup = text
    iframe = folium.IFrame(text, width=400*size, height=300*size,     ratio='70%')
    popup = folium.Popup(iframe,    max_width='100%')

    folium.Marker(locationlist[i], popup=popup, icon=folium.Icon(color='orange', icon='info-sign')).add_to(map2)
    
#     folium.CircleMarker(location=locationlist[i],
#                             radius= var[i] * factor,
#                             color="#007849",
#                             popup=string,
#                             fill=False).add_to(map2)
#     print(var[i])

# HeatMap(data=hospitais_assis[['lat','lng','var']].values.tolist(), radius=20, max_zoom=13).add_to(map2)

style = {'fillColor': '#00000000', 'color': '#366172'}
folium.GeoJson(sp_vale['geometry'], style_function=lambda x: style).add_to(map2)
    
# map2!

In [7]:
map2.save('map.html')
map2.save('../../sample_pages/images/emendas/map.html')